### Imports

In [ ]:
# Import os to manage paths
import os
# Import certifi to manage SSL certificates with TSL host
import certifi
# Import ConfigParser for handling config file
from configparser import ConfigParser
# Import PyMongo useful classes
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

### Read the configuration file and load the credentials

In [ ]:
# Read the configuration file with the credentials
config = ConfigParser()
config.read(os.path.join(os.getcwd(), 'config.ini'))

# Load the data to connect to MongoDB Atlas Cluster
user = config['MongoDB']['user']
password = config['MongoDB']['password']
uri = config['MongoDB']['uri']
db_name = config['MongoDB']['db_name']

### Connect to MongoDB Atlas Cluster

In [ ]:
# Generate the connection string for the client
CONNECTION_STRING = "mongodb+srv://{}:{}@{}/{}?retryWrites=true&w=majority"\
                    .format(user, password, uri, db_name)

# Connect to MongoDB Atlas Cluster
client = MongoClient(CONNECTION_STRING, 
                     server_api=ServerApi('1'),
                     tls=True,
                     tlsCAFile=certifi.where()
                    )

# Check if connection succeed
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")